In [1]:
#截圖、攝影機拍照，存成 Json

from PIL import ImageGrab, Image
import os
import random
from base64 import b64decode, b64encode
import pandas as pd
import numpy as np

# 開資料夾
if "temp" not in os.listdir():
    os.mkdir("temp")
    
if "data" not in os.listdir():
    os.mkdir("data")

#攝影機
#pip install opencv-python
from cv2 import VideoCapture, cvtColor, COLOR_RGB2BGR
cap = VideoCapture(0)

def getWebCamImg():
    ret, frame = cap.read()
    frame = cvtColor(np.array(frame), COLOR_RGB2BGR)
    return Image.fromarray(frame).resize((1280, 720), Image.ANTIALIAS)

def getScreenShot():
    screenShot = ImageGrab.grab()
    screenShot = screenShot.resize((1280, 720), Image.ANTIALIAS)
    return screenShot

def getImgB64encode(img):
    randomVal = random.getrandbits(32)
    filename = "./temp/%d.png"%randomVal
    img.save(filename)
    img_encode = b64encode(open(filename, 'rb').read())
    os.remove(filename)
    return img_encode.decode()

In [2]:
#彙整起來
def grabImgByFrame(timeFrame, recordID = "test"):
    if recordID not in os.listdir("./data/"):
        os.mkdir("./data/"+recordID)
    
    screenShot = getScreenShot()
    WebCam = getWebCamImg()

    outJson = {
        "webCam": getImgB64encode(WebCam),
        "screenShot": getImgB64encode(screenShot), 
    }

    import json

    timeFrame = 0
    filename = "./data/"+ recordID+"/"+ str(timeFrame) + ".json"


    with open(filename, 'w') as f:
        json.dump(outJson,f)
    
    return filename

# Flask API 的部分

In [7]:
import numpy as np
import pandas as pd
import json

from flask import Flask, request, abort
app = Flask(__name__)

from flask_cors import CORS
CORS(app)


@app.route("/start")
def start():
    #在這邊放，你讓程式開始紀錄的 Python 程式碼
    
    
    # print(request.cookies.get(WhatUwant)) #可以讀取cookies
    return json.dump({"status": "Good"})



@app.route("/getPrediction")
def get_prediction(user_id):
    return json.dump({"data": [0.1, 0.8, 0.1, 0.5, 0.6]}), 200

@app.route("/test")

def test():
    return str("Your test is successful.")

@app.route("/getImg")
def getImg():
    filename = grabImgByFrame(0)
    return json.dumps({"status": "Good", "filename":filename})

#uvicorn filename:app --port 8001 --workers 1 --proxy-headers
if __name__ == "__main__":
    app.run(host = '0.0.0.0', port = 13523)

ModuleNotFoundError: No module named 'flask_cors'

## 然後
只要在瀏覽器打上 http://localhost:13520 就可以呼叫API囉<br>
比方 http://localhost:13520/test 就會回應 "Your test is successful."<br>
http://localhost:13520/getImg 就會回應 Json 圖片檔案